In [ ]:
import pandas as pd
import os

# Directory where your folders are stored
directory = './datasets_full/'

# Create an empty list to store individual DataFrames
dfs = []

# Iterate over each folder in the directory
for year_folder in os.listdir(directory):
    year_folder_path = os.path.join(directory, year_folder)
    
    # Check if the item is a directory
    if os.path.isdir(year_folder_path):
        print(year_folder)
        # Iterate over each CSV file in the year folder
        for filename in os.listdir(year_folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(year_folder_path, filename)
                print("Reading file:", file_path)  # Print the file name

                # Read CSV file into DataFrame
                df = pd.read_csv(file_path, sep=';', encoding='latin1', decimal=',', low_memory=False)
                # Append DataFrame to list
                dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# Now you have a single DataFrame containing all data from CSV files in the folders

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
df.head(100)

Se verifican los tipos de datos antes que puedan influenciar a la hora de generar las diferentes tablas

In [ ]:
df.info()

Se cambian el tipo de valores para que estos no influyan en las tablas

In [ ]:
rowto_string = ['entCode','CodigoProductoONU','UnidadTiempoRenovacion']

df[rowto_string] = df[rowto_string].astype('string')

In [ ]:
df.info()

Tabla de datos antes de la limpieza, esta tabla es importante para conocer cuales pueden er los valores a limpiar además de conocer valores máximos o mínimos

In [ ]:
def describe_nulls(df):
    # Calcular estadísticas descriptivas
    describe_result = df.describe()

    # Contar valores nulos por columna
    nulos = df.isnull().sum()

    # Agregar la fila 'nulos' al resultado de describe()
    describe_result.loc['nulos'] = nulos

    # Calcular porcentaje de valores nulos por columna
    num_rows = len(df)
    null_percentages = (nulos / num_rows) * 100

    # Formatear los porcentajes con el símbolo de porcentaje
    null_percentages_formatted = null_percentages.apply(lambda x: f"{x:.2f}%")

    # Agregar la fila 'porcentaje de nulos' al resultado de describe()
    describe_result.loc['porcentaje de nulos'] = null_percentages_formatted

    return describe_result

def describe_nulls_num(df):
    # Calcular estadísticas descriptivas
    describe_result = df.describe(include=[object])

    # Contar valores nulos por columna
    nulos = df.isnull().sum()

    # Agregar la fila 'nulos' al resultado de describe()
    describe_result.loc['nulos'] = nulos

    # Calcular porcentaje de valores nulos por columna
    num_rows = len(df)
    null_percentages = (nulos / num_rows) * 100

    # Formatear los porcentajes con el símbolo de porcentaje
    null_percentages_formatted = null_percentages.apply(lambda x: f"{x:.2f}%")

    # Agregar la fila 'porcentaje de nulos' al resultado de describe()
    describe_result.loc['porcentaje de nulos'] = null_percentages_formatted

    return describe_result

In [ ]:
describe_nulls(df)

In [ ]:
describe_nulls_num(df)

En esta etapa se eliminan las columnas que tienen un alto valor de NA

In [ ]:
del_col = ['LicitacionBaseTipo', 'ContratoRenovable', 'UnidadTiempoRenovacion','FuenteFinanciamiento', 'FechaEntregaEnSoporteFisico', 'FechaEstimadaEvaluacionOfertas','ProhibicionSubContratacion', 'RazonPublicidadOfertasTecnicas', 'FechaEstimadaFirmaContrato', 'ObservacionContrato']
df_limpiado = df.drop(columns=del_col)

In [ ]:
df_limpiado.head()

In [ ]:
describe_nulls(df_limpiado)

In [ ]:
describe_nulls_num(df_limpiado)

In [ ]:
# Eliminar filas con valores nulos en una columna particular
nombre_columna = 'MonedaOferta'
df_sin_na = df_limpiado.dropna(subset=[nombre_columna])

In [ ]:
describe_nulls_num(df_sin_na)

In [ ]:
# Filtrar el DataFrame para obtener solo las filas donde el valor en la columna 'MontoTotalOferta' es mayor que 99
df_mayores_99 = df_sin_na[df_sin_na['MontoTotalOferta'] > 999]

# Contar los valores mayores a 99
conteo_mayores_99 = len(df_mayores_99)

# Contar los valores menores o iguales a 99
conteo_menores_igual_99 = len(df_sin_na) - conteo_mayores_99

# Imprimir los resultados
print("Número de valores mayores a 99:", conteo_mayores_99)
print("Número de valores menores o iguales a 99:", conteo_menores_igual_99)


In [ ]:
# Filtrar el DataFrame para obtener solo las filas donde el valor en la columna 'MontoTotalOferta' es mayor que 99
df_filtrado = df_sin_na[df_sin_na['MontoTotalOferta'] > 999]

In [ ]:
describe_nulls(df_filtrado)

In [ ]:
describe_nulls_num(df_filtrado)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
 
def remover_outliers_y_graficar(df, iteracion):
    # Graficar el boxplot antes de remover outliers
    plt.figure()
    sns.boxplot(x='MontoTotalOferta', data=df)
    plt.ticklabel_format(style='plain', axis='x')

    # Calcule el primer y tercer cuartil y la mediana después de remover outliers
    Q1 = df['MontoTotalOferta'].quantile(0.25)
    Q3 = df['MontoTotalOferta'].quantile(0.75)
    median = df['MontoTotalOferta'].median()

    # Calcule el rango intercuartílico (IQR)
    IQR = Q3 - Q1

    # Defina el límite inferior y superior para outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Identifique los outliers
    outliers = df[(df['MontoTotalOferta'] < limite_inferior) | (df['MontoTotalOferta'] > limite_superior)]
    df_limpiado = df.drop(outliers.index)

    # Obtener las posiciones de los cuartiles y la mediana en el eje x del boxplot
    cuartiles = df['MontoTotalOferta'].quantile([0.25, 0.75]).values
    median_pos = df['MontoTotalOferta'].median()

    # Agrega líneas verticales para los cuartiles y la mediana
    plt.axvline(cuartiles[0], color='r', linestyle='--', label='Cuartil 1 (Q1)')
    plt.axvline(cuartiles[1], color='g', linestyle='--', label='Cuartil 3 (Q3)')
    plt.axvline(median_pos, color='b', linestyle='-', label='Mediana')

    # Muestra los valores de los cuartiles y la mediana en el gráfico
    plt.text(cuartiles[0], 0, f'Q1 = {Q1:.2f}', rotation=0, va='top', ha='right', color='r', bbox=dict(facecolor='white', alpha=0.5))
    plt.text(cuartiles[1], 0, f'Q3 = {Q3:.2f}', rotation=0, va='top', ha='left', color='g', bbox=dict(facecolor='white', alpha=0.5))
    plt.text(median_pos, 0, f'Mediana = {median:.2f}', rotation=0, va='bottom', ha='center', color='b', bbox=dict(facecolor='white', alpha=0.5))

    # Muestra los cuartiles en la leyenda
    plt.legend()

    # Muestra información sobre la iteración y los outliers
    plt.title(f"Iteración {iteracion}: Nuevo rango [{df_limpiado['MontoTotalOferta'].min()}, {df_limpiado['MontoTotalOferta'].max()}], Outliers: {len(outliers)}")
    plt.show()

    return df_limpiado, outliers, df_limpiado


# Cargar el DataFrame
df_cuartiles = df_filtrado

# Iterar hasta que no haya más outliers
iteracion = 1
while True:
    df_cuartiles, outliers, df_actualizado = remover_outliers_y_graficar(df_cuartiles, iteracion)
    nuevo_rango_min = df_actualizado['MontoTotalOferta'].min()
    nuevo_rango_max = df_actualizado['MontoTotalOferta'].max()
    print(f"Iteración {iteracion}: Nuevo rango [{nuevo_rango_min}, {nuevo_rango_max}]")
    if outliers.empty:
        print("No se encontraron outliers en esta iteración.")
        break
    iteracion += 1


In [ ]:
describe_nulls(df_cuartiles)

In [114]:
describe_nulls_num(df_cuartiles)

,NroLicitacion,NombreLicitacion,TipoLicitacion,Descripcion,MonedaLicitacion,MontoEstimadoVisible,BaseEstimacionMontoLicitacion,JustificacionMontoEstimado,FechaPublicacion,FechaInicioPreguntas,FechaFinalPreguntas,FechaPublicacionRespuestas,FechaActoAperturaTecnica,FechaActoAperturaEconomica,FechaCierre,FechaAdjudicacion,UnidadTiempoEvaluacion,EstadoLicitacion,ContemplaObrasPublicas,LicitacionInformada,TipoAdjudicacion,TipoAprobacionAdjudicacion,NumeroActaAprobacion,FechaActaAprobacion,TipoConvocatoria,NroEtapasLicitacion,SubContratacion,TomaRazonContraloria,PublicidadOfertasTecnicas,Contrato,UnidadTiempoDuracionContrato,TipoEjecucion,PlazoPagoContrato,TipoPago,ExtensionPlazo,UnidadCompra,UnidadCompraRUT,Institucion,Sector,RubroN1,RubroN2,RubroN3,ONUProducto,NombreItem,DescripcionItem,UnidadMedida,Proveedor,ProveedorRUT,ActividadProveedor,TamanoProveedor,NombreOferta,EspecificacionesProveedor,EstadoOferta,MonedaOferta,ResultadoOferta
count,374994,374994,374994,374994,374994,374994,374994,232795,374994,374994,374994,374994,374994,374994,374994,374800,373837,374994,193786,374994,374994,374994,374988,374994,374994,374994,374994,347177,313359,347177,206022,188878,206022,193786,347175,374994,374994,374994,374994,374994,374994,374994,374994,374994,374716,374994,374994,374994,360714,374994,374994,374993,374994,374994,374994
unique,13739,13165,11,12879,5,2,3,2342,13162,12528,9916,9512,10442,10455,8347,13724,4,5,2,1,1,6,6965,1349,2,2,2,2,2,3,4,2,3,3,2,996,385,231,1,55,328,1465,6149,6199,113912,94,9028,9024,5680,5,30411,251768,2,5,2
top,4853-4-LP23,Adquisición de libros para Programa Bibliometro,Licitación Pública Menor a 100 UTM (L1),Adquisición de libros para puntos de préstamo ...,CLP,Si,Presupuesto disponible,0,08-08-2023 10:07:00,11-08-2023 19:51:00,14-08-2023 19:51:00,16-08-2023 12:00:00,29-08-2023 18:42:00,29-08-2023 18:42:00,28-08-2023 18:41:00,06-10-2023 16:19:52,Dias,Adjudicada,NO,NO,Multiple con Orden de Compra,Resolucion,1274,02-09-2015 0:00:00,Abierta,Una etapa,Si,No,Si,No,Horas,Contrato de Ejecucion Inmediata,A 30 dias,Cheque,NO,BIBLIOTECA CENTRAL UTALCA,60.905.000-4,DIRECCION GENERAL DE GENDARMERIA DE CHIL,"GOB. CENTRAL, UNIVERSIDADES",Productos impresos y publicaciones,Medios impresos,Publicaciones impresas,Libros de referencias para bibliotecas,Libros de referencias para bibliotecas,Adquisición de Insumos de Laboratorio para la ...,Unidad,DIAGONAL LIBROS LIMITADA,76.660.524-9,VENTA AL POR MAYOR DE LIBROS,Micro,COTIZACION DE LIBROS,Adjunto anexo económico.,Aceptada,CLP,Ganadora
freq,3813,8013,169010,3813,363996,201934,221430,92516,3813,3813,3813,3987,3813,3813,3813,3813,190256,373570,193680,374994,374994,325353,10227,12113,374665,368780,201934,346440,311927,189589,136642,132119,204848,102843,288058,12319,38129,63269,374994,82901,77381,77381,35183,35183,236,301624,10672,10672,18602,126550,4540,2969,348390,363996,224359
nulos,0,0,0,0,0,0,0,142199,0,0,0,0,0,0,0,194,1157,0,181208,0,0,0,6,0,0,0,0,27817,61635,27817,168972,186116,168972,181208,27819,0,0,0,0,0,0,0,0,0,278,0,0,0,14280,0,0,1,0,0,0
porcentaje de nulos,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,37.92%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.05%,0.31%,0.00%,48.32%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,7.42%,16.44%,7.42%,45.06%,49.63%,45.06%,48.32%,7.42%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.07%,0.00%,0.00%,0.00%,3.81%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%


In [ ]:
df_col_del = ['LicitacionBaseTipo', 'ContratoRenovable', 'UnidadTiempoRenovacion','FuenteFinanciamiento', 'FechaEntregaEnSoporteFisico', 'FechaEstimadaEvaluacionOfertas','ProhibicionSubContratacion', 'RazonPublicidadOfertasTecnicas', 'FechaEstimadaFirmaContrato', 'ObservacionContrato']
df_limpiado = df.drop(columns=df_col_del)